# OpenAI Function Calling In LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [16]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [17]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/home/centrox_ai/anaconda3/envs/llava/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [18]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [19]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

Note: The next cell is expected to generate an error.

In [20]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [21]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [22]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code']}}

In [23]:
from langchain.chat_models import ChatOpenAI

In [24]:
model = ChatOpenAI()

/home/centrox_ai/anaconda3/envs/llava/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [25]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd97efe30> JSON: {
  "prompt_tokens": 70,
  "completion_tokens": 17,
  "total_tokens": 87
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-acaa0f66-54d9-491d-a492-4fb953e43146-0')

In [26]:
model_with_function = model.bind(functions=[weather_function])

In [27]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd97ef8f0> JSON: {
  "prompt_tokens": 69,
  "completion_tokens": 17,
  "total_tokens": 86
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-82cbbe93-ccd4-415d-8c7f-ef7c5a7f71b9-0')

## Forcing it to use a function

We can force the model to use a function

In [28]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [29]:
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd97ee810> JSON: {
  "prompt_tokens": 79,
  "completion_tokens": 7,
  "total_tokens": 86
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-f6888e5a-0b18-4bbe-a86b-a8fffeae1ed7-0')

In [30]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"JFK"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd97eeb70> JSON: {
  "prompt_tokens": 74,
  "completion_tokens": 7,
  "total_tokens": 81
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-99096eee-9e60-46b4-9b05-6065a6575a52-0')

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [31]:
from langchain.prompts import ChatPromptTemplate

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [33]:
chain = prompt | model_with_function

In [34]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd957e690> JSON: {
  "prompt_tokens": 75,
  "completion_tokens": 17,
  "total_tokens": 92
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-5c55d137-053e-45ce-b94f-76c3924bde71-0')

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [35]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [36]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [37]:
model_with_functions = model.bind(functions=functions)

In [38]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd957db50> JSON: {
  "prompt_tokens": 116,
  "completion_tokens": 17,
  "total_tokens": 133
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-1ee26210-e574-4704-a30a-6d62de2ffcca-0')

In [39]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{"artist_name":"Taylor Swift","n":3}'}}, response_metadata={'token_usage': <OpenAIObject at 0x757bd957e630> JSON: {
  "prompt_tokens": 118,
  "completion_tokens": 21,
  "total_tokens": 139
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-687940f8-914a-42ba-ade1-931074afb918-0')

In [40]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': <OpenAIObject at 0x757bd957dc70> JSON: {
  "prompt_tokens": 111,
  "completion_tokens": 10,
  "total_tokens": 121
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_a450710239', 'finish_reason': 'stop', 'logprobs': None}, id='run-bba6d824-4c2f-44fa-b44c-a06f58279e41-0')